# Collect Satellite images of the farmers location from multiple timestamps

In [ ]:
from python_scripts.connector import *
from python_scripts.helper import *

from eval_scripts.evalscript_true_color import *
from eval_scripts.evalscript_ndvi import *
from eval_scripts.evalscript_ndvi_values  import *

from sentinelhub import MimeType, CRS, BBox, SentinelHubRequest, SentinelHubDownloadClient,DataCollection, bbox_to_dimensions, SHConfig
from sentinelhub.geo_utils import to_wgs84
from sentinelhub.constants import CRS
from sentinelhub import SentinelHubCatalog

import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


## Step 1: Get the necessary timestamps

In [ ]:
def get_time_slots(start, end, chunks):
    """
    Get the avalable time windows (time slots)
    """ 
    datetime_start = datetime.datetime.strptime(start, '%Y-%m-%d')
    datetime_end   = datetime.datetime.strptime(end, '%Y-%m-%d')
    
    tdelta = (datetime_end - datetime_start) / chunks
    edges = [(datetime_start + i*tdelta).date().isoformat() for i in range(chunks)]
    slots = [(edges[i], edges[i+1]) for i in range(len(edges)-1)]
    return slots
    

In [ ]:
slots = get_time_slots('2017-01-01', '2022-01-01', 5)
print("Yearly time windows")
for slot in slots:
    print(slot)

## Step 2: Get the necessary coordinates 

In [ ]:
# convert coordinations_file (csv file) into a dataframe
coordinations_df = pd.read_excel('data/locations/FarmerLocationExtract4Interns_sentinel-hub.xlsx')

In [ ]:
# Based on how far wail is with the conversion of the coordinates of the farmers location, we need to check whether or 
# the coordination needs to mdofied. 
coordinations_df

In [ ]:
coordinations_df.columns

In [ ]:
def bbox_converter(x):
    """
    Convert coordinates (longitude, latitude) in to a bbox in WGS84 format.
    
    return [long, lat, long, lat]
    """
    lng, lat = to_wgs84(x['P1_Longitude'], x['P1_Latitude'], CRS.WGS84)
    coords = [lng -  0.025, lat - 0.025, lng +  0.022, lat + 0.022]
    return coords

In [ ]:
geolocations  = coordinations_df[['P1_Longitude', 'P1_Latitude']].copy()

In [ ]:
geolocations[geolocations['P1_Latitude'].str.contains('"')]

In [ ]:
geolocations['P1_Latitude'] = geolocations['P1_Latitude'].str.replace('"', '')

In [ ]:
geolocations[geolocations['P1_Latitude'].str.contains('"')]

In [ ]:
geolocations = geolocations.astype("float64")

In [ ]:
geolocations.dtypes

In [ ]:
geolocations.iloc[:3]

In [ ]:
# geolocations.to_csv('coordinaten.csv', sep='\t')
geolocations.iloc[:15]

## Download all the satellite images 

In [ ]:
import time # time the performance
import os, shutil

In [ ]:
def least_clouds():
    return { 
    "dataFilter": { 
        "maxCloudCoverage": 0
        } 
    } 

In [ ]:
def empty_folder(folder):
    """
    delete files in folder 
    """
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
    

In [ ]:
empty_folder('../data/images')
empty_folder('../data/ndvi_values_mask')

In [ ]:
def get_request_(evalscript, time_interval, coords, data_folder):
    return SentinelHubRequest(
        data_folder=data_folder, 
        evalscript=evalscript, 
        input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L2A, 
            time_interval=time_interval,
            other_args = least_clouds())], 
        bbox=BBox(coords,crs=CRS.WGS84), 
        config=config, 
        size=[512, 512], 
        responses=[SentinelHubRequest.output_response('default', MimeType.PNG)])
    
    

In [ ]:
from eval_scripts.evalscript_true_color import *
from eval_scripts.evalscript_forest import *
from eval_scripts.evalscript_ndvi import *

In [ ]:
def download_images(geolocations, evalscript, slots, data_folder):
    for index, row in geolocations.iterrows(): # iterate through each location
        # create a list of requests
        coords = bbox_converter(row) # convert each location to a bbox
        print(coords)

        list_of_requests = [get_request_(evalscript,slot, coords,data_folder) for slot in slots]
        list_of_requests = [request.download_list[0] for request in list_of_requests]
        
        # download data with multiple threads
        SentinelHubDownloadClient(config=config).download(list_of_requests, max_threads=8)

In [ ]:
start = time.perf_counter()
download_images(geolocations.iloc[:3],evalscript_true_color, slots, '../data/images_completed_png' )
end = time.perf_counter()
print("Done in: {} seconds".format( end - start))

In [ ]:
# dowload mask - duurt heel lang
start = time.perf_counter()
download_images(geolocations.iloc[:10],evalscript_ndvi, slots, '../data/mask_completed' )
end = time.perf_counter()
print("Done in: {} seconds".format( end - start))

In [ ]:
from PIL import Image

In [ ]:
def download_mask_v2(geolocations, slots, data_folder):
    for index, rows in geolocations.iterrows(): 
        coords = bbox_converter(rows) # convert each location to a bbox
        for slot in slots:
            img_ndvi = sentinel_request(evalscript_ndvi_values, coords, slot, config, False, data_folder, other_args=least_clouds())
            min_ndvi = 0.6
            ndvi_copy = img_ndvi.copy()
            labels = np.where(ndvi_copy > min_ndvi, 255, 0)
            labels = labels.astype(np.uint8)
            img = Image.fromarray(labels)
            filename = "{}_{}_{}_{}_{}_{}".format(coords[0], coords[1], coords[2],coords[3], slot[0], slot[1])
            img.save(f'{data_folder}/{filename}.png')

In [ ]:
start = time.perf_counter()
download_mask_v2(geolocations.iloc[:30], slots,'../data/ndvi_values_mask')
end = time.perf_counter()
print("Done in: {} seconds".format( end - start))

## Get and rename images

In [ ]:
import os
import json
import shutil

In [ ]:
def parse_json(path):
    input_json = open(path)
    pased_json = json.load(input_json)
    return pased_json

In [ ]:
def build_filename(json):

    item =  json['payload']['input']
    bbox = item['bounds']['bbox']
    date_start= item['data'][0]['dataFilter']['timeRange']['from'].split("T")[0]
    date_end  = item['data'][0]['dataFilter']['timeRange']['to'].split("T")[0]
    
    return "{}_{}_{}_{}_{}_{}".format(bbox[0], bbox[1], bbox[2],bbox[3], date_start, date_end) # TODO : check format and . in naamgeving 

In [ ]:
#build_filename(parse_json(r'C:\Users\Amaryllis Lee\FARM-deforestation\data\images\1484b4269e568648b673959b5a955ce4\request.json'))

In [ ]:
def images_structure(rootdir,new_rootdir):
    for root, subdirectories, files in os.walk(rootdir):
        for subdirectory in subdirectories: # for each folder in dir:
            request_json = f'{rootdir}/{subdirectory}/request.json'
            response_png = f'{rootdir}/{subdirectory}/response.png'
            if os.path.isfile(request_json) and os.path.isfile(response_png):
                json_file = parse_json(request_json)
                new_filename = f'{new_rootdir}/{build_filename(json_file)}.png'
                shutil.copyfile(response_png,new_filename)
            # get image:
            #rename image
            #move to data amsk aand remove folder/subdir
            

In [ ]:
empty_folder('../data/images_completed_png')

In [ ]:
rootdir_true_colors = '../data/images_completed_png'
new_rootdir_true_colors = '../data/images_completed_png'
images_structure(rootdir_true_colors, new_rootdir_true_colors)

In [ ]:
# Modifying filename move mask to mask_png_testset
# rootdir_mask = '../data/mask'
# new_rootdir_mask= '../data/mask_png_testset'
# images_structure(rootdir, new_rootdir)